# 📘 Clase 20: Rutas y Metodos HTTP

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/heldigard/unaula-IF0100-POO-II/blob/main/notebooks/unidad-03/clase-20-rutas-http.ipynb)

## 🎯 Objetivos de Aprendizaje

Al finalizar esta clase, seras capaz de:
- Usar todos los metodos HTTP (GET, POST, PUT, PATCH, DELETE)
- Definir parametros de ruta y query
- Manejar status codes HTTP
- Crear estructura de rutas RESTful
- Usar routers de FastAPI

---

## 📚 Teoria: Metodos HTTP

| Metodo | Uso | Idempotente |
|--------|-----|-------------|
| **GET** | Obtener recursos | ✅ Si |
| **POST** | Crear recursos | ❌ No |
| **PUT** | Actualizar completo | ✅ Si |
| **PATCH** | Actualizar parcial | ❌ No |
| **DELETE** | Eliminar recursos | ✅ Si |

---

## 🔗 Parametros de Ruta vs Query

In [ ]:
# ============================================
# PARAMETROS EN FASTAPI
# ============================================

CODIGO = '''
from fastapi import FastAPI, Path, Query
from typing import Optional

app = FastAPI()

# ========== PARAMETROS DE RUTA ==========
# Forman parte de la URL: /items/42

@app.get("/items/{item_id}")
async def get_item(item_id: int):
    return {"item_id": item_id}

# Validacion de parametros de ruta
@app.get("/items/{item_id}")
async def get_item_validated(
    item_id: int = Path(..., gt=0, description="ID del item")
):
    return {"item_id": item_id}

# ========== PARAMETROS DE QUERY ==========
# Despues del ?: /items/?skip=0&limit=10

@app.get("/items/")
async def list_items(
    skip: int = 0,
    limit: int = Query(10, le=100),
    search: Optional[str] = None
):
    return {
        "skip": skip,
        "limit": limit,
        "search": search
    }

# ========== COMBINACION ==========
@app.get("/users/{user_id}/items/{item_id}")
async def get_user_item(
    user_id: int,
    item_id: int,
    q: Optional[str] = None
):
    return {
        "user_id": user_id,
        "item_id": item_id,
        "query": q
    }
'''

print(CODIGO)

---

## 📝 CRUD Completo

In [ ]:
# ============================================
# API CRUD COMPLETA
# ============================================

CRUD_API = '''
from fastapi import FastAPI, HTTPException, status
from typing import List, Optional
from pydantic import BaseModel

app = FastAPI()

# Modelo de datos
class Item(BaseModel):
    id: Optional[int] = None
    name: str
    description: Optional[str] = None
    price: float
    is_active: bool = True

class ItemUpdate(BaseModel):
    name: Optional[str] = None
    description: Optional[str] = None
    price: Optional[float] = None
    is_active: Optional[bool] = None

# Base de datos simulada
items_db = []
id_counter = 1

# ========== CREATE ==========
@app.post("/items", response_model=Item, status_code=status.HTTP_201_CREATED)
async def create_item(item: Item):
    global id_counter
    item.id = id_counter
    id_counter += 1
    items_db.append(item)
    return item

# ========== READ (LIST) ==========
@app.get("/items", response_model=List[Item])
async def list_items(
    skip: int = 0,
    limit: int = 10,
    active_only: bool = False
):
    result = items_db
    if active_only:
        result = [i for i in result if i.is_active]
    return result[skip : skip + limit]

# ========== READ (DETAIL) ==========
@app.get("/items/{item_id}", response_model=Item)
async def get_item(item_id: int):
    for item in items_db:
        if item.id == item_id:
            return item
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Item {item_id} no encontrado"
    )

# ========== UPDATE (FULL) ==========
@app.put("/items/{item_id}", response_model=Item)
async def update_item(item_id: int, item: Item):
    for idx, existing in enumerate(items_db):
        if existing.id == item_id:
            item.id = item_id
            items_db[idx] = item
            return item
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Item {item_id} no encontrado"
    )

# ========== UPDATE (PARTIAL) ==========
@app.patch("/items/{item_id}", response_model=Item)
async def patch_item(item_id: int, item_update: ItemUpdate):
    for existing in items_db:
        if existing.id == item_id:
            update_data = item_update.dict(exclude_unset=True)
            for field, value in update_data.items():
                setattr(existing, field, value)
            return existing
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Item {item_id} no encontrado"
    )

# ========== DELETE ==========
@app.delete("/items/{item_id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_item(item_id: int):
    for idx, item in enumerate(items_db):
        if item.id == item_id:
            items_db.pop(idx)
            return
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail=f"Item {item_id} no encontrado"
    )
'''

print(CRUD_API)

---

## 🗂️ Routers

In [ ]:
# ============================================
# ORGANIZACION CON ROUTERS
# ============================================

ROUTERS_CODE = '''
# routers/items.py
from fastapi import APIRouter, HTTPException, status

router = APIRouter(
    prefix="/items",           # Prefijo para todas las rutas
    tags=["items"],            # Tag para documentacion
    responses={404: {"description": "Not found"}}
)

@router.get("/")
async def list_items():
    return {"items": []}

@router.get("/me")
async def get_my_items():
    return {"my_items": []}

@router.get("/active")
async def get_active_items():
    return {"active_items": []}

# main.py
from fastapi import FastAPI
from routers import items, users

app = FastAPI()

# Incluir routers
app.include_router(items.router)
app.include_router(users.router)

# Las rutas seran:
# GET /items/
# GET /items/me
# GET /items/active
'''

print(ROUTERS_CODE)

---

## 📝 Ejercicio: API de Tareas

Crea una API REST completa para gestionar tareas con:
- CRUD completo
- Filtros por estado (pendiente, completada)
- Busqueda por titulo

In [ ]:
# Tu implementacion aqui
print('✅ Implementa la API de Tareas')

---

**¡Construye APIs RESTful profesionales! 🔗**